# Counter

Every time the singleton is spent, the count is incresed by one.

The observer observes the blockchain and check when there is a coin with key/values, `["Counter", "A"]`. The observer can monitor when a singleton is spent and check the value from the parent's solution.

In [1]:
# import singleton_helpers
import sys
sys.path.insert(0, ".")

from pathlib import Path

import singleton_helpers

In [2]:
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet
from cdv.util.load_clvm import load_clvm
from clvm_tools.clvmc import compile_clvm_text

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

counter_clsp = Path('singleton_counter.clsp').read_text()

counter_puzzle = Program(
    compile_clvm_text(counter_clsp, search_paths=["../include"])
)

inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(counter_puzzle)

standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [("Counter", "A")]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')

# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

alice: 2000000000000

starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [3]:
# spend eve singleton
network.sim.pass_blocks(1)
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin = await singleton_helpers.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id) 

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)

current_value = 0 # eve singleton has zero value
inner_solution = Program.to([
    singleton_puzzle.get_tree_hash(), 
    singleton_coin.amount, 
    inner_puzzle.get_tree_hash(), 
    current_value + 1
])

singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}]')

print('\nsingleton_coin_spend:')
singleton_helpers.print_json(singleton_coin_spend.to_json_dict())

from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
await network.push_tx(spend_bundle)

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
ffa098a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50aff8203ffffa01194ba575907e87f7478167d938824ce215eb3a906384e07e3bc74b3d98913e5ff0180]

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
        "puzzle_hash": "0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080

{'additions': [Coin(parent_coin_info=<bytes32: 319563d316c50eb4e8323a5a90c8db6a579802c694503f0232355b7270e5b4bc>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)]}

In [4]:
async def get_counter_value(get_coin_records_by_parent_ids, get_coin_record_by_name, get_puzzle_and_solution, launcher_id):
    singleton_coin = await singleton_helpers.get_unspent_singleton(get_coin_records_by_parent_ids, launcher_id)
    if singleton_coin == None:
        return None
    else:
        # get solution of the spent parent
        parent_id = singleton_coin.parent_coin_info
        parent_coin_record = await get_coin_record_by_name(parent_id)
        spent_block_index = parent_coin_record.spent_block_index
        coin_spent = await get_puzzle_and_solution(parent_id, spent_block_index)
        solutions = list(coin_spent.solution.to_program().as_iter())
        solution = list(solutions)[2]
        current_value = int.from_bytes((solution.as_atom_list())[3], byteorder='little')
        return current_value

In [5]:
# spend another singleton to increase value

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(singleton_coin_spend)

singleton_coin = await singleton_helpers.get_unspent_singleton(get_coin_records_by_parent_ids, launcher_id)
current_value = await get_counter_value(get_coin_records_by_parent_ids, get_coin_record_by_name, get_puzzle_and_solution, launcher_id)
inner_solution = Program.to([
    singleton_puzzle.get_tree_hash(), 
    singleton_coin.amount, 
    inner_puzzle.get_tree_hash(), 
    current_value + 1
])

singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}]')

print('\nsingleton_coin_spend:')
singleton_helpers.print_json(singleton_coin_spend.to_json_dict())
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
await network.push_tx(spend_bundle)

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x319563d316c50eb4e8323a5a90c8db6a579802c694503f0232355b7270e5b4bc',
 'puzzle_hash': '0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0x1194ba575907e87f7478167d938824ce215eb3a906384e07e3bc74b3d98913e5',
 'parent_name': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570'}

inner_solution:
ffa098a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50aff8203ffffa01194ba575907e87f7478167d938824ce215eb3a906384e07e3bc74b3d98913e5ff0280]

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x319563d316c50eb4e8323a5a90c8db6a579802c694503f0232355b7270e5b4bc",
        "puzzle_hash": "0x98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04

{'additions': [Coin(parent_coin_info=<bytes32: f25b047a7b830e74747caeea9a752f93af748101e0523b2947fdcd15c8d76726>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 319563d316c50eb4e8323a5a90c8db6a579802c694503f0232355b7270e5b4bc>, puzzle_hash=<bytes32: 98a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a>, amount=1023)]}

In [51]:
from clvm_tools.binutils import disassemble, assemble

# the observer looks for a coin with ["Counter", "A"]
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals
block_records = await get_block_records(0, 20)
header_hashes = list(map(lambda br: br.header_hash, block_records))
for header_hash in header_hashes:
    _, removals = await get_additions_and_removals(header_hash)
    launchers = list(filter(lambda cr: cr.coin.puzzle_hash == singleton_top_layer.SINGLETON_LAUNCHER_HASH, removals))
    for cr in launchers:
        coin_id = cr.coin.name()
        coin_spent = await get_puzzle_and_solution(coin_id, cr.spent_block_index)
        solutions = list(coin_spent.solution.to_program().as_iter())
        print(solutions)
        kv = coin_spent.solution.to_program().at("rrff")
        if kv == assemble('("Counter" . "A")'):
            print("found counter A")



[Program(a098a9cac73d70ccf45a659858f390cac158e764cf864d168841173c22c862a50a), Program(8203ff), Program(ffff87436f756e7465724180)]
found counter A
